# Trends in Number of COVID-19 and Vaccine (Github)

## Call package

In [6]:
import pandas as pd
import numpy as np
import datetime

In [7]:
# Path of the data's location
path = '/Users/donkim/Desktop/python-dashboard/Python COVID-19 and Vaccination Dashboard/'

## Data

In [8]:
covid_conf = pd.read_csv(path + 'time_series_covid19_confirmed_US.csv')
covid_deat = pd.read_csv(path + 'time_series_covid19_deaths_US.csv')
covid_pop  = pd.read_csv(path + 'time_series_covid19_vaccine_doses_admin_US.csv')
covid_vacc = pd.read_csv(path + 'people_vaccinated_us_timeline.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/donkim/Desktop/python-dashboard/Python COVID-19 and Vaccination Dashboard/time_series_covid19_confirmed_US.csv'

### Covid-19

In [ ]:
### COVID-19 (Confirmed by State)
# 1) column's drop
# 2) filter: iso2 == 'US'
# 3) transform: wide → long
# 4) date type: m/d/y → yyyy-mm-dd
# 5) sum: US (group by state & date)
# 6) filter: date ~ 2021-10-31 
covid_conf = covid_conf.drop(['UID', 'iso3', 'code3', 'FIPS', 'Admin2','Country_Region', 'Lat', 'Long_', 'Combined_Key'], axis=1)
conf = covid_conf[covid_conf['iso2'] == 'US']
conf = pd.melt(conf, id_vars = ['iso2', 'Province_State'], var_name='date', value_name='confirmed')
conf['date'] = pd.to_datetime(conf.date)
conf = conf.groupby(['Province_State', 'date'], as_index = False).sum()  # US (group by date).
conf = conf[conf['date'] <= '2021-10-31']
print(conf.head(), '\n', conf.tail())

In [4]:
### COVID-19 (Deaths by State)
# 1) column's drop
# 2) filter: iso2 == 'US'
# 3) transform: wide → long
# 4) date type: m/d/y → yyyy-mm-dd
# 5) sum: US (group by state & date)
# 6) filter: date ~ 2021-10-31
covid_deat = covid_deat.drop(['UID', 'iso3', 'code3', 'FIPS', 'Admin2','Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population'], axis=1)
deat = covid_deat[covid_deat['iso2'] == 'US']
deat = pd.melt(deat, id_vars = ['iso2', 'Province_State'], var_name='date', value_name='deaths')
deat['date'] = pd.to_datetime(deat.date)
deat = deat.groupby(['Province_State', 'date'], as_index = False).sum()  # US (group by date).
deat = deat[deat['date'] <= '2021-10-31']
print(deat.head(), '\n', deat.tail())

NameError: name 'covid_deat' is not defined

In [5]:
### Join (Confirmed & Deaths) and rename
covid = pd.merge(conf, deat, how='left', on=['Province_State', 'date'])
covid.columns = ['state', 'date', 'confirmed', 'deaths']
print(covid.head(), '\n', covid.tail())

NameError: name 'conf' is not defined

### Population

In [7]:
### US Population
# 1) filter: iso2 == 'US'
# 2) column's select
# 3) rename
covid_pop = covid_pop[covid_pop['iso2'] == 'US']
pop = covid_pop[['Province_State', 'Population']]
pop.columns = ['state', 'population']
print(pop.head(), '\n', pop.tail())

        state  population
0     Alabama   4903185.0
1      Alaska    731545.0
2     Arizona   7278717.0
3    Arkansas   3017804.0
4  California  39512223.0 
             state  population
46       Virginia   8535519.0
47     Washington   7614893.0
48  West Virginia   1792147.0
49      Wisconsin   5822434.0
50        Wyoming    578759.0


### Vaccines

In [8]:
### COVID-19 (Vaccines)
################################### Fully= fully vaccinated, Fully+Partial = at least 1
# 1) filter: iso2 == 'US'
# 2) column's drop
# 3) date type: m/d/y → yyyy-mm-dd
# 4) sum
# 5) rename
# 6) filter: date ~ 2021-10-31
vacc = covid_vacc.drop(['FIPS', 'Country_Region', 'Lat', 'Long_', 'Combined_Key'], axis=1)
vacc['Date'] = pd.to_datetime(vacc.Date)
vacc['at_least_one'] = vacc['People_Fully_Vaccinated'] + vacc['People_Partially_Vaccinated']
vacc = vacc.drop(['People_Partially_Vaccinated'], axis=1)
vacc.columns = ['state', 'date', 'fully', 'at_least_one']
vacc = vacc[vacc['date'] <= '2021-10-31']
print(vacc.head(), '\n', vacc.tail())

     state       date  fully  at_least_one
0  Alabama 2020-12-10    NaN           NaN
1  Alabama 2020-12-11    NaN           NaN
2  Alabama 2020-12-12    NaN           NaN
3  Alabama 2020-12-13    NaN           NaN
4  Alabama 2020-12-14    NaN           NaN 
          state       date     fully  at_least_one
21047  Wyoming 2021-10-27  252966.0      292654.0
21048  Wyoming 2021-10-28  253414.0      293260.0
21049  Wyoming 2021-10-29  253866.0      293892.0
21050  Wyoming 2021-10-30  253968.0      294093.0
21051  Wyoming 2021-10-31  254028.0      294284.0


## Final Data Set (Join)

In [9]:
# 1) Join by state (Pop + Vacc)
vacc = pd.merge(pop, vacc, how='left', on=['state'])

# 2) Join by state & date
df = pd.merge(covid, vacc, how='left', on=['state', 'date'])
print(df.head(), '\n', df.tail())

     state       date  confirmed  deaths  population  fully  at_least_one
0  Alabama 2020-01-22          0       0         NaN    NaN           NaN
1  Alabama 2020-01-23          0       0         NaN    NaN           NaN
2  Alabama 2020-01-24          0       0         NaN    NaN           NaN
3  Alabama 2020-01-25          0       0         NaN    NaN           NaN
4  Alabama 2020-01-26          0       0         NaN    NaN           NaN 
          state       date  confirmed  deaths  population     fully  \
34392  Wyoming 2021-10-27     101912    1174    578759.0  252966.0   
34393  Wyoming 2021-10-28     102403    1174    578759.0  253414.0   
34394  Wyoming 2021-10-29     102926    1174    578759.0  253866.0   
34395  Wyoming 2021-10-30     102926    1174    578759.0  253968.0   
34396  Wyoming 2021-10-31     102926    1174    578759.0  254028.0   

       at_least_one  
34392      292654.0  
34393      293260.0  
34394      293892.0  
34395      294093.0  
34396      294284.0  


In [10]:
df.head()

,state,date,confirmed,deaths,population,fully,at_least_one
0,Alabama,2020-01-22,0,0,NaN,NaN,NaN
1,Alabama,2020-01-23,0,0,NaN,NaN,NaN
2,Alabama,2020-01-24,0,0,NaN,NaN,NaN
3,Alabama,2020-01-25,0,0,NaN,NaN,NaN
4,Alabama,2020-01-26,0,0,NaN,NaN,NaN


In [11]:
df.tail()

,state,date,confirmed,deaths,population,fully,at_least_one
34392,Wyoming,2021-10-27,101912,1174,578759.0,252966.0,292654.0
34393,Wyoming,2021-10-28,102403,1174,578759.0,253414.0,293260.0
34394,Wyoming,2021-10-29,102926,1174,578759.0,253866.0,293892.0
34395,Wyoming,2021-10-30,102926,1174,578759.0,253968.0,294093.0
34396,Wyoming,2021-10-31,102926,1174,578759.0,254028.0,294284.0


In [18]:
df.to_csv(path + 'covid_vaccine.csv', index=False)

In [ ]:
### 2021-10-31 (End) ***** Update